In [1]:
import scipy.linalg as la
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
from helper_functions.grape_functions import get_state_index
from main_grape.Grape import Grape
from IPython import display
import numpy as np
%pylab inline


Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [ ]:
total_time = 50.0
steps = 1000



#Defining H0
qubit_state_num = 4

#Defining H0
alpha = 0.224574
freq_ge = 4.9#GHz
ens_1 = np.array([ 2*np.pi*ii*(freq_ge - 0.5*(ii-1)*alpha) for ii in np.arange(qubit_state_num)])
H0_1 = np.diag(ens_1)

alpha = 0.224574
freq_ge = 4.5#GHz
ens_2 = np.array([ 2*np.pi*ii*(freq_ge - 0.5*(ii-1)*alpha) for ii in np.arange(qubit_state_num)])
H0_2 = np.diag(ens_2)

mode_state_num = 3
qm_g1 =2*np.pi*0.1
mode_freq = 6.102 #GHz
mode_ens = np.array([ 2*np.pi*ii*(mode_freq) for ii in np.arange(mode_state_num)])
qm_g2 = 2*np.pi*0.1
#mode_freq2 = 5.327 #GHz
state_num = mode_state_num * (qubit_state_num**2)
Q_x   = np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)+np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1)
Q_y   = (0+1j) *(np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)-np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1))
Q_z   = np.diag(np.arange(0,qubit_state_num))

M_x = np.diag(np.sqrt(np.arange(1,mode_state_num)),1)+np.diag(np.sqrt(np.arange(1,mode_state_num)),-1)

I_q = np.identity(qubit_state_num)
I_m = np.identity(mode_state_num)
H_q_1 = np.diag(ens_1-ens_1[0])
H_q_2 = np.diag(ens_2-ens_2[0])
H_m = np.diag(mode_ens)

H0 = np.kron(H_q_1,np.kron(I_q,I_m)) + np.kron(I_q,np.kron(H_q_2,I_m)) +\
        np.kron(I_q,np.kron(I_q,H_m)) + qm_g1*np.kron(Q_x,np.kron(I_q,M_x)) +\
        qm_g2*np.kron(I_q,np.kron(Q_x,M_x))

#Defining Dressed information for our multimode exp        
w_c, v_c = la.eig(H0)
dressed=[]
D= False
if D:
    for ii in range (len(v_c)):
        index=np.argmax(np.abs(v_c[:,ii]))
        if index not in dressed:
            dressed.append(index)
        else:
            temp= (np.abs(v_c[:,ii])).tolist()
            while index in dressed:

                temp.remove(max(temp))
                index2= np.argmax(np.array(temp))

                if index2<index:
                            #dressed.append(index2)
                    index=index2
                else:
                            #dressed.append(index2-1)
                    index=index2+1
            dressed.append(index)




#Defining Concerned states (starting states)
psi0 = [0,mode_state_num,qubit_state_num* mode_state_num,(qubit_state_num+1)* mode_state_num ] #gg0,ge0,eg0,ee0

#Defining states to include in the drawing of occupation
states_draw_list = psi0
states_draw_names = ['gg0','ge0','eg0','ee0']

first_hs = range((qubit_state_num-1)*(qubit_state_num)*mode_state_num , state_num)
second_hs = range((qubit_state_num-1)*mode_state_num , (qubit_state_num)*mode_state_num)
for ii in range (qubit_state_num-2):
    index = ((qubit_state_num-1)*mode_state_num) + (ii+1) *qubit_state_num* mode_state_num
    second_hs = second_hs + (range(index , index + qubit_state_num-1))
forbidden = first_hs +second_hs

for ii in range(state_num):
    if (ii % 3 == 2) and ii not in forbidden:
        forbidden.append(ii)
print forbidden
#Defining U (Target)
ee0=get_state_index((qubit_state_num+1)* mode_state_num,dressed)
eg0=get_state_index(qubit_state_num* mode_state_num,dressed)

print dressed
print ee0,eg0

if D:
    Ct_c= np.identity(state_num) - np.outer(v_c[:,ee0], np.conjugate((v_c[:,ee0])))
    Ct_c= Ct_c - np.outer(v_c[:,eg0], np.conjugate((v_c[:,eg0])))
    Ct_c= Ct_c + np.outer(v_c[:,ee0], np.conjugate((v_c[:,eg0])))
    Ct_c= Ct_c + np.outer(v_c[:,eg0], np.conjugate((v_c[:,ee0])))
else:
    #Ut_c = la.expm((0-1j)*total_time*H0)
    #Ct_c = Ut_c.copy()
    Ct_c = np.identity(state_num)
    Ct_c[ee0,eg0] = 1
    Ct_c[eg0,ee0] = 1
    Ct_c[ee0,ee0] = 0
    Ct_c[eg0,eg0] = 0
U = Ct_c

#Defining U0 (Initial)
q_identity = np.identity(qubit_state_num)
m_identity = np.identity(mode_state_num)

#Defining control Hs
XI_1 = np.kron(Q_x,np.kron(I_q,I_m))
ZI_1 = np.kron(Q_z,np.kron(I_q,I_m))
YI_1 = np.kron(Q_y,np.kron(I_q,I_m))
XI_2 = np.kron(I_q,np.kron(Q_x,I_m))
ZI_2 = np.kron(I_q,np.kron(Q_z,I_m))
YI_2 = np.kron(I_q,np.kron(Q_y,I_m))
Hops = [XI_1,YI_1,ZI_1, XI_2,YI_2,ZI_2]
ops_max_amp = [6.0,6.0,6.0,6.0,6.0,6.0]
Hnames =['x1','y1','z1','x2','y2','z2']

#Defining convergence parameters
max_iterations = 5000
decay = max_iterations/2
convergence = {'rate':0.002, 'update_step':50, 'max_iterations':max_iterations,\
               'conv_target':1e-8,'learning_rate_decay':decay}
penalty_coeffs = {'envelope' : 0, 'dc':0, 'dwdt':0.0001,'d2wdt2':0, 'forbidden':10.0}
#penalty_coeffs = {'envelope' : 0, 'dc':0, 'dwdt':0,'d2wdt2':0, 'forbidden':0}
multi_mode = {'dressed':dressed, 'vectors':v_c, 'qnum':qubit_state_num, 'mnum': mode_state_num,\
              'f':freq_ge, 'es':w_c, 'g1':qm_g1, 'D':D, 'Interpolation':False, 'Modulation':False}


print psi0
print H_q_1
print H_q_2
print H_m

[36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 9, 10, 11, 21, 22, 23, 33, 34, 35, 2, 5, 8, 14, 17, 20, 26, 29, 32]
[]
15 12
[0, 3, 12, 15]
[[  0.           0.           0.           0.        ]
 [  0.          30.78760801   0.           0.        ]
 [  0.           0.          60.16417595   0.        ]
 [  0.           0.           0.          88.12970384]]
[[  0.           0.           0.           0.        ]
 [  0.          28.27433388   0.           0.        ]
 [  0.           0.          55.13762771   0.        ]
 [  0.           0.           0.          80.58988148]]
[[  0.           0.           0.        ]
 [  0.          38.33999674   0.        ]
 [  0.           0.          76.67999349]]


In [ ]:
 #Extra options
#reg_coeffs = reg_coeffs,draw = [states_draw_list,states_draw_names] ,forbidden = states_forbidden_list,multi_mode = multi_mode,
#maxA = ops_max_amp,use_gpu = True
uks,U_f = Grape(H0,Hops,Hnames,U,total_time,steps,psi0,convergence, draw = [states_draw_list,states_draw_names],show_plots = True,
                maxA = ops_max_amp, 
       multi_mode=multi_mode, penalty_coeffs = penalty_coeffs, forbidden = forbidden,
      method = 'L-BFGS-B', switch =True)

Using 5 Taylor terms and 7 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagators initialized.
Intermediate propagators correlations initialized.
Vectors initialized.
Training loss initialized.